# 古诗生成水墨画示例

本notebook演示如何使用该项目从古诗生成水墨画。

## 1. 环境设置

首先导入必要的库，并设置必要的路径。

In [ ]:
import os
import sys
import torch
from PIL import Image
import matplotlib.pyplot as plt

# 添加项目根目录到Python路径
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.utils.utils import load_config, make_image_grid
from src.inference.generate import load_model, generate_image

## 2. 配置

加载配置文件并设置相关参数。

In [ ]:
# 加载配置
config_path = os.path.join(project_root, 'config/train_config.yaml')
config = load_config(config_path)

# 设置GPU或CPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"使用设备: {device}")

# 设置模型路径 - 请根据您的实际路径修改
model_path = os.path.join(project_root, 'models/checkpoints/final')

# 检查模型路径是否存在
if not os.path.exists(model_path):
    print(f"警告: 模型路径不存在: {model_path}，请先训练模型或修改路径")

## 3. 加载模型

加载预训练的Stable Diffusion模型，或者您的微调模型。

In [ ]:
# 如果模型不存在，可以使用默认的SD模型进行演示
if not os.path.exists(model_path):
    print("使用默认的Stable Diffusion模型进行演示")
    model_path = config["model"]["pretrained_model_name_or_path"]

# 加载模型
pipeline = load_model(model_path, config, device)

## 4. 从古诗生成水墨画

现在我们使用加载的模型，从古诗生成水墨画。

In [ ]:
# 示例古诗
poetry_samples = [
    "山中夜坐，北风吹雨，叶漏声疏，漏声迟，惊顾枕上，时闻雨声。",
    "春眠不觉晓，处处闻啼鸟。夜来风雨声，花落知多少。",
    "两个黄鹂鸣翠柳，一行白鹭上青天。窗含西岭千秋雪，门泊东吴万里船。",
    "江南好，风景旧曾谙。日出江花红胜火，春来江水绿如蓝。能不忆江南？"
]

# 设置随机种子以便结果可复现
seed = 42

In [ ]:
# 为每首诗生成图像
all_images = []
for i, poetry in enumerate(poetry_samples):
    print(f"\n生成第 {i+1} 首诗的水墨画: {poetry}")
    images, prompt = generate_image(pipeline, poetry, num_images=1, seed=seed + i)
    all_images.extend(images)
    
    # 显示生成的图像
    plt.figure(figsize=(6, 6))
    plt.imshow(images[0])
    plt.title(f"诗: {poetry[:20]}..." if len(poetry) > 20 else f"诗: {poetry}")
    plt.axis('off')
    plt.show()

## 5. 显示图像网格

将生成的所有图像组合成一个网格展示。

In [ ]:
# 创建2x2的图像网格
grid_image = make_image_grid(all_images, 2, 2)

# 显示网格图像
plt.figure(figsize=(12, 12))
plt.imshow(grid_image)
plt.title("古诗生成的水墨画作品")
plt.axis('off')
plt.show()

## 6. 保存结果

将生成的图像保存到文件。

In [ ]:
# 创建输出目录
output_dir = os.path.join(project_root, 'data/output')
os.makedirs(output_dir, exist_ok=True)

# 保存单张图像
for i, image in enumerate(all_images):
    output_path = os.path.join(output_dir, f"poem_{i+1}.png")
    image.save(output_path)
    print(f"已保存图像: {output_path}")

# 保存网格图像
grid_path = os.path.join(output_dir, "poetry_grid.png")
grid_image.save(grid_path)
print(f"已保存网格图像: {grid_path}")

## 7. 自定义生成

您可以在下面输入您自己的古诗，生成对应的水墨画。

In [ ]:
# 输入您自己的古诗
your_poetry = "" # 在这里输入您的古诗

if your_poetry:
    # 生成多张图像并比较
    images, prompt = generate_image(pipeline, your_poetry, num_images=4, seed=seed)
    
    # 创建2x2的图像网格
    grid_image = make_image_grid(images, 2, 2)
    
    # 显示网格图像
    plt.figure(figsize=(12, 12))
    plt.imshow(grid_image)
    plt.title(f"您的古诗: {your_poetry[:30]}..." if len(your_poetry) > 30 else f"您的古诗: {your_poetry}")
    plt.axis('off')
    plt.show()
    
    # 保存结果
    custom_path = os.path.join(output_dir, "custom_poetry_grid.png")
    grid_image.save(custom_path)
    print(f"已保存您的自定义古诗生成结果: {custom_path}")